## Run CNMF source extraction on movies
Step 2 of the Caiman processing pipeline for dendritic two-photon calcium imaging movies. This part uses mmap files as input. These are created during motion correction with the Caiman toolbox (see `01_Preprocess_MotionCorrect.ipynb`). 

### Imports & Setup
The first cells import the various Python modules required by the notebook. In particular, a number of modules are imported from the Caiman package. In addition, we also setup the environment so that everything works as expected.

In [ ]:
# Generic imports
# from __future__ import absolute_import, division, print_function
# from builtins import *

import os, platform, glob, sys, re
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat

from IPython.display import clear_output

%matplotlib inline

In [ ]:
# on Linux we have to add the caiman folder to Pythonpath
if platform.system() == 'Linux':
    sys.path.append(os.path.expanduser('~/caiman'))
# environment variables for parallel processing
os.environ['MKL_NUM_THREADS']='1'
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['VECLIB_MAXIMUM_THREADS']='1'

In [ ]:
# CaImAn imports
import caiman as cm
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.components_evaluation import estimate_components_quality as estimate_q
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.source_extraction.cnmf import utilities as cnmf_utils

### Select files and parameters
The following need to be specified:
- data_folder ... location of the data (relative to ~/Data)
- mc_output ... select if output of rigid ('rig') or piece-wise rigid ('els') motion correction should be used (currently only 'rig' is tested and works)
- max_files ... maximum number of files to process, e.g. for testing (if 0, all files will be processed)

In [ ]:
animal_folder = 'M1_for_processing'
date_folder = 'M1_2018-01-31'
spot_folder = 'S1'

mc_output = 'rig'
remove_bad_frames = True # remove bad frames specified in Json file

# create the complete path to the data folder
if platform.system() == 'Linux':
    data_folder = '/home/ubuntu/Data/Henry_test'
elif platform.system() == 'Darwin':
    data_folder = '/Users/Henry/polybox/Data_temp/Dendrites_Gwen'
data_folder = os.path.join(data_folder, animal_folder, date_folder, spot_folder)
data_folder

In [ ]:
# select the mmap file created during motion correction
all_files = os.listdir(data_folder)
matches = [x for x in all_files if x.startswith('%s_%s' % (date_folder, spot_folder)) 
           and x.endswith('.mmap') and mc_output in x]

if len(matches) == 1: # if only 1 file matches, select it
    fname = matches[0]
    print('Selected file: %s' % (fname))
elif len(matches) > 0: # if several files match, ask which one to choos
    print('Found several matching files!')
    for ix, match in enumerate(matches):
        print('%1.0f: %s' % (ix+1,match))
    select = raw_input('Please select your file (1-%1.0f):' % (ix+1))
    fname = matches[int(select)-1]
    print('Selected file: %s' % (fname))
else: # if no file matches, throw an error
    raise Exception('Did not find a matching mmap file!')

fname = os.path.join(data_folder, fname)

# get metadata for corresponding joined file
meta = json.load(open(fname[:fname.find('_crop_')+5] + '.json'))

### Setup cluster
The default backend mode for parallel processing is through the multiprocessing package. This will allow us to use all the cores in the VM.

In [ ]:
# start the cluster (if a cluster already exists terminate it)
n_processes = 8 # number of compute processes (None to select automatically)
if 'dview' in locals():
    dview.terminate()
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=n_processes, single_thread=False)

### Parameters for source extraction
Next, we define the important parameters for calcium source extraction. These parameters will have to be iteratively refined for the respective datasets.

The parameters are stored in the Python dictionary `params_cnmf`.

In [ ]:
# parameters for source extraction and deconvolution
p = 1                       # order of the autoregressive system (normally 1, 2 for fast indicators / imaging)
gnb = 2                     # number of global background components
merge_thresh = 0.8          # merging threshold, max correlation allowed
rf = None                   # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50 / None: no patches
stride_cnmf = 6             # amount of overlap between the patches in pixels
K = 20                      # number of components per patch
gSig = [4, 4]               # expected half size of neurons
init_method = 'sparse_nmf'  # initialization method (if analyzing dendritic data using 'sparse_nmf')
is_dendrites = True         # flag for analyzing dendritic data
#alpha_snmf = 10e2           # sparsity penalty for dendritic data analysis through sparse NMF
alpha_snmf = 1e-6

method_deconvolution='oasis'# deconvolution method (oasis or cvxpy)
final_rate = 13.1316             # final frame rate in Hz


# parameters for component evaluation
# min_SNR = 2.5               # signal to noise ratio for accepting a component
# rval_thr = 0.8              # space correlation threshold for accepting a component
# cnn_thr = 0.8               # threshold for CNN based classifier

In [ ]:
# store parameters in Python dictionary
params = dict()
params['p'] = p
params['gnb'] = gnb
params['merge_thresh'] = merge_thresh
params['rf'] = rf
params['stride_cnmf'] = stride_cnmf
params['K'] = K
params['gSig'] = gSig
params['init_method'] = init_method
params['is_dendrites'] = is_dendrites
params['alpha_snmf'] = alpha_snmf

params['method_deconvolution'] = method_deconvolution
params['final_rate'] = final_rate

params

### Run CNMF on joined mmap file
According to the Caiman tutorials, CNMF source extraction should be run in several stages. First, we run CNMF on patches of the dataset and then evaluate the quality of the extracted components. Next, CNMF is run again but on the full field-of-view. Finally, the extracted components are again evaluated and classified (good and bad).

Load mmap file.

In [ ]:
Yr, dims, T = cm.load_memmap(fname)

In [ ]:
# remove bad frames specified in corresponding Json file
# Todo: print deleted frames
if remove_bad_frames:
    bad_frames = json.load(open(fname.replace('.mmap','badFrames.json')))
    bad_frames = np.array(bad_frames['frames'])
    Yr = np.delete(Yr, bad_frames, axis=1)
    T = Yr.shape[1]
    images = np.reshape(Yr.T, [T] + list(dims), order='F')
    fname_new = cm.save_memmap([images], base_name=os.path.join(data_folder, 'removedFrames'))
    Yr, dims, T = cm.load_memmap(fname_new)
    print('Deleted %1d frames. Saved to new file %s.' % (len(bad_frames), os.path.basename(fname_new)))
    print('Deleted frames:')
    print(bad_frames)

In [ ]:
d1, d2 = dims
images = np.reshape(Yr.T, [T] + list(dims), order='F')
Y = np.reshape(Yr, dims + (T,), order='F')
m_images = cm.movie(images)
#  checks on movies (might take time if large!)
if np.min(images) < 0:
    raise Exception('Movie too negative, add_to_movie should be larger')
if np.sum(np.isnan(images)) > 0:
    raise Exception('Movie contains nan! You did not remove enough borders')
# correlation image
Cn = cm.local_correlations(Y)
Cn[np.isnan(Cn)] = 0

In [ ]:
# sanity check the image
avg_img = np.mean(images,axis=0)
plt.figure(figsize=(10,20))
plt.subplot(1,2,1)
plt.imshow(avg_img, cmap='gray'), plt.title('Frame average')
plt.subplot(1,2,2)
plt.imshow(Cn, cmap='jet'), plt.title('Correlation image')

### Run single CNMF on full FoV, without patches

In [ ]:
# Configure CNMF and fit to the data
cnmf_single = cnmf.CNMF(n_processes, k=params['K'], gSig=params['gSig'], merge_thresh=params['merge_thresh'], 
                p=params['p'], dview=dview, Ain=None, gnb=params['gnb'], rf=params['rf'],
                memory_fact=1, method_init=params['init_method'], alpha_snmf=params['alpha_snmf'],
                method_deconvolution=params['method_deconvolution'])
cnmf_single = cnmf_single.fit(images)

In [ ]:
# Unravel results
A, C, b, f, YrA, sn = cnmf_single.A, cnmf_single.C, cnmf_single.b, cnmf_single.f, cnmf_single.YrA, cnmf_single.sn
# A   ... n_pixel x n_components sparse matrix (component locations)
# C   ... n_component x t np.array (fitted signal)
# b   ... ? np.array
# f   ... ? np.array (b / f related to global background components)
# YrA ... n_component x t np.array (residual)
# sn  ... n_pixel np.array (SNR?)

print('Detected components: %1.0f' % (C.shape[0]))

Evaluate components (discard low quality components).

In [ ]:
r_values_min = .65         # threshold on space consistency
fitness_min = -100        # threshold on time variability
fitness_delta_min = -100  # threshold on time variability (if nonsparse activity)
Npeaks = 100
thresh_C = 0.3
Athresh = 0.1

traces = C + YrA
idx_comps, idx_comps_bad, fitness_raw, fitness_delta, r_values = estimate_q(traces, Y, A, C, b, f, final_frate=params['final_rate'], 
                                      Npeaks=Npeaks, r_values_min=r_values_min, fitness_min=fitness_min, 
                                      fitness_delta_min=fitness_delta_min, thresh_C=thresh_C, Athresh=Athresh,
                                      return_all=True)
print('Good / bad components: %1.0f / %1.0f' % (len(idx_comps), len(idx_comps_bad)))

In [ ]:
# Plot good and bad components
plt.figure(figsize=(20,30));
plt.subplot(121); crd_good = cm.utils.visualization.plot_contours(A[:,idx_comps], Cn, thr=.8, vmax=0.75)
plt.title('Contour plots of accepted components')
plt.subplot(122); crd_bad = cm.utils.visualization.plot_contours(A[:,idx_comps_bad], Cn, thr=.8, vmax=0.75)
plt.title('Contour plots of rejected components')

In [ ]:
# Plot good components on background image and as component map
A_dense = A.todense()
counter = 1
plt.figure(figsize=(20,20));
for i_comp in range(len(idx_comps)):
    plt.subplot(len(idx_comps),2,counter)
    counter += 1
    dummy = cm.utils.visualization.plot_contours(A[:,idx_comps[i_comp]], avg_img, cmap='gray', 
                                                 colors='r', display_numbers=False)
    component_img = np.array(np.reshape(A_dense[:,idx_comps[i_comp]], dims, order='F'))
    plt.subplot(len(idx_comps),2,counter)
    counter += 1
    plt.imshow(component_img), plt.title('Component %1.0f' % (i_comp))
    
    if i_comp == 0:
        component_matrix = component_img
    else:
        component_matrix = np.dstack((component_matrix, component_img))
plt.tight_layout()

### Save and view results

In [ ]:
# saving
np.savez(os.path.join(os.path.split(fname_new)[0],
                      os.path.split(fname_new)[1][:-5] + 'results_CNMF.npz'),
         Cn=Cn, A=A.todense(), C=C,
         b=b, f=f, YrA=YrA, sn=sn, d1=d1, d2=d2,
         idx_components=idx_comps, idx_components_bad=idx_comps_bad)

Interactive viewer for traces of accepted and rejected components
**Currently does not work remotely!**

In [ ]:
# accepted components
if len(idx_comps) > 0:
    nb_view_patches(Yr, A.tocsc()[:, idx_comps], C[idx_comps], 
                    b, f, dims[0], dims[1], YrA=YrA[idx_comps], image_neurons = Cn,
                    denoised_color = 'red');
else:
    print("No accepted components!")

### Calculate DFF and plot traces
The CaImAn function `detrend_df_f` uses a sliding window percentile filter to determine the baseline and compute DFF. 

In [ ]:
F_dff = cnmf_utils.detrend_df_f(A, b, C, f, YrA = YrA, quantileMin=8, frames_window=100)
# select good components
F_dff = F_dff[idx_comps,:]

In [ ]:
# zoom on time axis
# set either one to -1 for full range
t1 = -1 # in s
t2 = -1

fig = plt.figure(figsize=(20,10)) # increase figsize to (20,10) to improve visibility
# loop over good components to plot their respective DFF timeseries
t = np.arange(0, F_dff.shape[-1]) / params['final_rate']

if t1 > 0 and t2 > 0:
    t1_frame = np.where(t>=t1)[0][0]
    t2_frame = np.where(t<=t2)[0][-1]
    t = t[t1_frame:t2_frame]
    F_dff_plot = F_dff[:,t1_frame:t2_frame]
else:
    F_dff_plot = F_dff.copy()

offset = 0
for i_comp in range(len(idx_comps)):
    plot_trace = F_dff_plot[i_comp, :]
    plot_trace = plot_trace - min(plot_trace) + offset
    offset = max(plot_trace)
    plt.plot(t, plot_trace)

# some improvements to the default figure
plt.xlabel('Time [s]', fontsize=18)
plt.ylabel('DFF', fontsize=18)
plt.ylim((0, offset)), plt.xlim((np.min(t), np.max(t)))
plt.title('%s %s CNMF Results' % (date_folder, spot_folder), fontsize=20)
plt.legend(['Component %1d' % (x) for x in range(len(idx_comps))])
ax = fig.gca()
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)

# save the figure if required
# (extension determines file type)
if t1 > 0 and t2 > 0:
    figfile_name = '%s_%s_CNMF_results_%1d-%1ds.png' % (date_folder, spot_folder, t1, t2)
else:
    figfile_name = '%s_%s_CNMF_results.png' % (date_folder, spot_folder)
plt.savefig(os.path.join(data_folder, figfile_name))

### Split up by trials and save as .mat

In [ ]:
# split up F_dff by trials
# Todo: correct source_frames: key
source_files = meta['source_file']
source_frames = np.array(meta['source_frames'])

In [ ]:
# check if our numbers match
if not (np.sum(source_frames)-len(bad_frames)) == F_dff.shape[-1]:
    raise Exception('Sum of source frames minus number of bad frames must be equal to number of timepoints.')

In [ ]:
results_dff = dict()
removed_frames = dict()
bad_frames_removed = []
start_frame = 0
for ix, trial_file in enumerate(source_files):
    trial_indices = list(range(start_frame, source_frames[ix] + start_frame))
#     print(trial_file)
#     print('Start / stop: %1.0f / %1.0f' % (trial_indices[0], trial_indices[-1]))
    
    # check if trial contains bad frames (that have not been removed yet)
    bad = [x for x in bad_frames if x in trial_indices and x not in bad_frames_removed]
    
    # remove indices of bad frames
    if len(bad) > 0:
#         print('Found %1.0f bad frames' % (len(bad)))
        del trial_indices[-len(bad):]
#         print('Removed frames (relative to trial start):')
        for i_bad in bad:
            bad_frames_removed.append(i_bad)
        removed_frames_trial = [x-trial_indices[0] for x in bad]
    else:
        removed_frames_trial = []
    
#     print('Start / stop: %1.0f / %1.0f\n' % (trial_indices[0], trial_indices[-1]))
    
    # create a valid Matlab variable / field name
    field_name = str('x' + source_files[ix]).replace('.tif','').replace('-','_')
    results_dff[field_name] = F_dff[:,trial_indices]
    removed_frames[field_name] = removed_frames_trial
    
    start_frame = trial_indices[-1] + 1

In [ ]:
# prepare the dictionary for saving as mat file
# the field names will be the variable names in Matlab
mdict={'trials': [str(x) for x in source_files], 
       'dff_trial': results_dff, 
       'removed_frames': removed_frames,
       'mean_image': avg_img,
       'spatial_components': component_matrix
      }

In [ ]:
# save the .mat file
matfile_name = '%s_%s_CNMF_results.mat' % (date_folder, spot_folder)
savemat(os.path.join(data_folder, matfile_name), mdict=mdict)

### Run double CNMF first on patches, then on full FoV
This is the recommended approach according to the tutorial notebooks.

In [ ]:
# for this step deconvolution is turned off (p=0)
cnmf_patch = cnmf.CNMF(n_processes, k=params['K'], gSig=params['gSig'], merge_thresh=params['merge_thresh'], 
                p=0, dview=dview, Ain=None, rf=params['rf'], stride=params['stride_cnmf'], 
                memory_fact=1, method_init=params['init_method'], alpha_snmf=params['alpha_snmf'],
                only_init_patch=True, gnb=params['gnb'], method_deconvolution=params['method_deconvolution'])
cnmf_patch = cnmf_patch.fit(images)

A_tot, C_tot, YrA_tot, b_tot, f_tot, sn_tot = \
cnmf_patch.A, cnmf_patch.C, cnmf_patch.YrA, cnmf_patch.b, cnmf_patch.f, cnmf_patch.sn

#clear_output()

Evaluate components (discard low quality components).

In [ ]:
r_values_min = .7          # threshold on space consistency
fitness_min = -40          # threshold on time variability
fitness_delta_min = -40    # threshold on time variability (if nonsparse activity)
Npeaks = 100
traces = C_tot + YrA_tot
idx_comps, idx_comps_bad = estimate_q(traces, Y, A_tot, C_tot, b_tot, f_tot, final_frate=params['final_rate'], 
                                      Npeaks=Npeaks, r_values_min=r_values_min, fitness_min=fitness_min, 
                                      fitness_delta_min=fitness_delta_min)

Plot contours of selected and rejected components

In [ ]:
# Plot components
plt.figure(figsize=(10,20));
plt.subplot(121); crd_good = cm.utils.visualization.plot_contours(A_tot[:,idx_comps], Cn, thr=.8, vmax=0.75)
plt.title('Contour plots of accepted components')
plt.subplot(122); crd_bad = cm.utils.visualization.plot_contours(A_tot[:,idx_comps_bad], Cn, thr=.8, vmax=0.75)
plt.title('Contour plots of rejected components')

Run CNMF on full field of view.

In [ ]:
cnmf_full = cnmf.CNMF(n_processes, k=A_tot.shape[-1], gSig=params['gSig'], merge_thresh=params['merge_thresh'], 
                p=params['p'], dview=dview, Ain=A_tot, Cin=C_tot, f_in=f_tot, rf=params['rf'],
                memory_fact=1, method_init=params['init_method'], alpha_snmf=params['alpha_snmf'],
                stride=params['stride_cnmf'], method_deconvolution=params['method_deconvolution'])
cnmf_full = cnmf_full.fit(images)
A, C, b, f, YrA, sn = cnmf_full.A, cnmf_full.C, cnmf_full.b, cnmf_full.f, cnmf_full.YrA, cnmf_full.sn

#clear_output()

Recheck quality of components with stricter criteria.

In [ ]:
r_values_min = .7         # threshold on space consistency
fitness_min = -100        # threshold on time variability
fitness_delta_min = -100  # threshold on time variability (if nonsparse activity)
Npeaks = 100
thresh_C = 0.3
Athresh = 0.1

traces = C + YrA
idx_comps, idx_comps_bad = estimate_q(traces, Y, A, C, b, f, final_frate=params['final_rate'], 
                                      Npeaks=Npeaks, r_values_min=r_values_min, fitness_min=fitness_min, 
                                      fitness_delta_min=fitness_delta_min, thresh_C=thresh_C, Athresh=Athresh)

Plot contours of selected and rejected components

In [ ]:
# Plot components
plt.figure(figsize=(10,20));
plt.subplot(121); crd_good = cm.utils.visualization.plot_contours(A[:,idx_comps], Cn, thr=.8, vmax=0.75)
plt.title('Contour plots of accepted components')
plt.subplot(122); crd_bad = cm.utils.visualization.plot_contours(A[:,idx_comps_bad], Cn, thr=.8, vmax=0.75)
plt.title('Contour plots of rejected components')

### Stop cluster and clean up log files

In [ ]:
cm.stop_server(dview=dview)
log_files = glob.glob('*_LOG_*')
for log_file in log_files:
    os.remove(log_file)

### Save and view results

In [ ]:
# saving
np.savez(os.path.join(os.path.split(fname)[0],
                      os.path.split(fname)[1][:-4] + 'results_CNMF.npz'),
         Cn=Cn, A=A.todense(), C=C,
         b=b, f=f, YrA=YrA, sn=sn, d1=d1, d2=d2,
         idx_components=idx_comps, idx_components_bad=idx_comps_bad)

Interactive viewer for traces of accepted and rejected components
**Currently does not work remotely!**

In [ ]:
# accepted components
if len(idx_comps) > 0:
    nb_view_patches(Yr, cnmf_obj2.A.tocsc()[:, idx_comps], cnmf_obj2.C[idx_comps], 
                    cnmf_obj2.b, cnmf_obj2.f, dims[0], dims[1], YrA=cnmf_obj2.YrA[idx_comps], image_neurons = Cn,
                    denoised_color = 'red');
else:
    print("No accepted components!")

In [ ]:
# rejected components
if len(idx_comps_bad) > 0:
    nb_view_patches(Yr, cnmf_obj2.A.tocsc()[:, idx_comps_bad], cnmf_obj2.C[idx_comps_bad], 
                    cnmf_obj2.b, cnmf_obj2.f, dims[0], dims[1], YrA=cnmf_obj2.YrA[idx_comps_bad], image_neurons = Cn,
                    denoised_color = 'red');
else:
    print("No rejected components!")

Reconstruct denoised movie.

In [ ]:
denoised = cm.movie(cnmf_obj2.A.dot(cnmf_obj2.C) + \
                    cnmf_obj2.b.dot(cnmf_obj2.f)).reshape(dims + (-1,), order='F').transpose([2, 0, 1])

Play original and denoised data side-by-side.

In [ ]:
m_orig = cm.load(fname_new)

In [ ]:
# play along side original data
cm.concatenate([m_orig, denoised], axis=2).play(fr=params['final_rate'], 
                                                gain=15, magnification=2, offset=0, backend='notebook')

# Old code
Code below is kept from previous versions of the notebook. **Do NOT use!!!**

In [ ]:
# create list of folders for processing
experiment_folders = []
session = []
spot = []
for session_folder in os.listdir(data_folder):
    if os.path.isdir(os.path.join(data_folder, session_folder)):
        if re.match(r'(\S{2})_(\d{4})-(\d{2})-(\d{2})', session_folder):
            spot_counter = 0
            for spot_folder in os.listdir(os.path.join(data_folder, session_folder)):
                if os.path.isdir(os.path.join(data_folder, session_folder, spot_folder)) \
                and re.match(r'S[1-9]', spot_folder):
                    experiment_folders.append(os.path.join(data_folder, session_folder, spot_folder))
                    session.append(session_folder)
                    spot.append(spot_folder)
        else:
            print('Skipping %s' % (os.path.join(data_folder, session_folder)))
experiment_folders

### Create joined mmap file from single experiments
Next, we join individual experiments into a big mmap file. Calcium source extraction will be run on the joined mmap file. The joined mmap file (`Join_*.mmap`) is saved in the data folder together with a metadata file (`Join_*.json`) which has information about the contained experiments.

In [ ]:
if len(mmap_files)>1:
    fname_new = cm.save_memmap_join(mmap_files, base_name='Join', n_chunks=12, dview=dview)
    # move the file to the enclosing folder
    clear_output()
    print('\nCreated joined mmap file\n%s' % (fname_new))
    # create a Json file with information about source files
    meta = {"joined": fname_new, "source": mmap_files}
    json_fname = fname_new.replace('.mmap','.json')
    with open(json_fname, 'w') as fid:
        json.dump(meta, fid)
    print('\nCreated meta file\n%s' % (json_fname))
else:
    print('One file only, not re-saving!')
    fname_new = mmap_files[0]